In [1]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense,CuDNNLSTM, Dropout, Activation
from keras.layers.normalization import BatchNormalization
from keras.optimizers import RMSprop
from janome.tokenizer import Tokenizer
import numpy as np
import random
import glob
import sys
import io
import re
import os

t = Tokenizer()

Using TensorFlow backend.


In [2]:
char_indices = {}  # 辞書初期化
indices_char = {}  # 逆引き辞書初期化
texts= []
chars = []
parts = []
paths = glob.glob('C:/Users/hata/sozoenshu/small_data/*.txt')
for path in paths:
    word = []
    part = []
    print('reading ' , path)
    binarydata = open(path,'rb').read()
    text = binarydata.decode('shift_jis')
    text = re.split(r'\-{5,}',text)[2]
    text = re.split(r'底本：',text)[0]
    text = text.replace('|','')
    text = text.replace('\u3000','')
    text = re.sub(r'《.+?》','',text) # ルビをとる
    text = re.sub(r'※','',text) # ※をとる
    text = re.sub(r'sentimentalisme','センチメンタリズム',text)
    text = re.sub(r'\r\n','',text)
    text = re.sub(r'―','',text)
    text = re.sub(r'［＃.+?］','',text) # 入力注をとる
    texts.append(text)
    print('corpus length:', len(text))
    malist = t.tokenize(text)
    for w in malist:
        word.append(w.surface)
        part.append(w.part_of_speech.split(','))
    chars.append(word)
    parts.append(part)
    next_PoS = {}
    # 次に来る品詞を保存しておく
    for i in range(len(malist)-1):
        w = malist[i]
        if w.part_of_speech in next_PoS:
            if malist[i+1].part_of_speech not in next_PoS[w.part_of_speech]:
                next_PoS[w.part_of_speech].append(malist[i+1].part_of_speech)
        else:
            next_PoS[w.part_of_speech] = [malist[i+1].part_of_speech]

POS = []
for i in range(len(parts)):
    for j in range(len(parts[i])):
        if parts[i][j] not in POS:
            POS.append(parts[i][j])

print('Part of Speech:',POS)
print('Part of Speech length:', len(POS))
print('texts length:',len(texts))
print('chars length:', len(chars))

# 辞書の作成
global_chars = []
for i in range(len(chars)):
    print('chars['+str(i)+']:', len(chars[i]))
    for word in chars[i]:
        if not word in global_chars:  # 未登録なら
            global_chars.append(word)  # 登録する

char_indices = dict((c,i) for i,c in enumerate(global_chars))
indices_char = dict((i,c) for i,c in enumerate(global_chars))

POS_indices = dict((str(c),i) for i,c in enumerate(POS))
indices_POS = dict((i,str(c)) for i,c in enumerate(POS))

print('global_chars length', len(global_chars))

del text
del texts
del paths

reading  C:/Users/hata/sozoenshu/small_data\susanono_mikoto.txt
corpus length: 42370
Part of Speech: [['名詞', '数', '*', '*'], ['名詞', '一般', '*', '*'], ['助詞', '連体化', '*', '*'], ['助詞', '係助詞', '*', '*'], ['助詞', '格助詞', '一般', '*'], ['動詞', '自立', '*', '*'], ['助動詞', '*', '*', '*'], ['記号', '句点', '*', '*'], ['名詞', '副詞可能', '*', '*'], ['助詞', '接続助詞', '*', '*'], ['記号', '読点', '*', '*'], ['動詞', '非自立', '*', '*'], ['形容詞', '自立', '*', '*'], ['名詞', '接尾', '助数詞', '*'], ['名詞', '形容動詞語幹', '*', '*'], ['連体詞', '*', '*', '*'], ['接頭詞', '名詞接続', '*', '*'], ['副詞', '一般', '*', '*'], ['名詞', '非自立', '助動詞語幹', '*'], ['名詞', '固有名詞', '地域', '一般'], ['名詞', '接尾', '一般', '*'], ['助詞', '並立助詞', '*', '*'], ['名詞', '非自立', '副詞可能', '*'], ['副詞', '助詞類接続', '*', '*'], ['名詞', '固有名詞', '人名', '名'], ['名詞', '代名詞', '一般', '*'], ['動詞', '接尾', '*', '*'], ['助詞', '副詞化', '*', '*'], ['接続詞', '*', '*', '*'], ['名詞', '固有名詞', '地域', '国'], ['助詞', '副助詞', '*', '*'], ['名詞', '非自立', '一般', '*'], ['助詞', '副助詞／並立助詞／終助詞', '*', '*'], ['名詞', 'サ変接続', '*', '*'], ['助詞', '格助詞', '連語', '

In [8]:
next_PoS = {}
for i in range(len(malist)-1):
    w = malist[i]
    if w.part_of_speech in next_PoS:
        if malist[i+1].part_of_speech not in next_PoS[w.part_of_speech]:
            next_PoS[w.part_of_speech].append(malist[i+1].part_of_speech)
    else:
        next_PoS[w.part_of_speech] = [malist[i+1].part_of_speech]

In [12]:
for i in next_PoS.keys():
    print(i)
    print(next_PoS[i])
    print()

名詞,数,*,*
['名詞,一般,*,*', '名詞,接尾,助数詞,*', '名詞,数,*,*', '連体詞,*,*,*', '動詞,自立,*,*', '助詞,連体化,*,*', '名詞,固有名詞,地域,一般', '名詞,副詞可能,*,*', '接続詞,*,*,*', '副詞,一般,*,*', '名詞,非自立,副詞可能,*', '記号,一般,*,*', '助詞,格助詞,一般,*', '記号,括弧開,*,*', '名詞,代名詞,一般,*', '名詞,接尾,一般,*']

名詞,一般,*,*
['助詞,連体化,*,*', '助詞,係助詞,*,*', '助詞,格助詞,一般,*', '形容詞,自立,*,*', '動詞,自立,*,*', '名詞,接尾,一般,*', '助詞,並立助詞,*,*', '名詞,一般,*,*', '助詞,副助詞,*,*', '助動詞,*,*,*', '助詞,副助詞／並立助詞／終助詞,*,*', '助詞,格助詞,連語,*', '動詞,非自立,*,*', '副詞,一般,*,*', '名詞,接尾,副詞可能,*', 'フィラー,*,*,*', '名詞,固有名詞,人名,名', '名詞,サ変接続,*,*', '接続詞,*,*,*', '副詞,助詞類接続,*,*', '記号,読点,*,*', '名詞,接尾,形容動詞語幹,*', '接頭詞,数接続,*,*', '名詞,非自立,副詞可能,*', '助詞,終助詞,*,*', '助詞,格助詞,引用,*', '名詞,代名詞,一般,*', '連体詞,*,*,*', '名詞,非自立,一般,*', '名詞,数,*,*', '名詞,接尾,人名,*', '記号,句点,*,*', '記号,一般,*,*', '名詞,接尾,助数詞,*', '名詞,接尾,サ変接続,*', '名詞,形容動詞語幹,*,*', '接頭詞,名詞接続,*,*', '動詞,接尾,*,*', '名詞,副詞可能,*,*', '名詞,固有名詞,一般,*', '名詞,接尾,地域,*', '名詞,固有名詞,地域,一般', '形容詞,接尾,*,*', '助詞,副詞化,*,*']

助詞,連体化,*,*
['名詞,一般,*,*', '接頭詞,名詞接続,*,*', '名詞,非自立,副詞可能,*', '名詞,副詞可能,*,*', '名詞,非自立,助動詞語幹,*', '助動詞,*,*,*',